In [2]:
%%capture --no-stderr
!pip install langgraph langchain-openai langchain-community langchain-core tavily-python wikipedia


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Setup - STEP 1

In [3]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [5]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "stock-market-analyzer"

In [6]:
_set_env("TAVILY_API_KEY")

## Define analyst - STEP 2

In [7]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

class FinancialAnalyst(BaseModel):
    affiliation: str = Field(
        description="Primary affiliation of the analyst (e.g., Investment Bank, Hedge Fund).",
    )
    name: str = Field(
        description="Name of the analyst."
    )
    role: str = Field(
        description="Role of the analyst (e.g., Technical Analyst, Fundamental Analyst, Sentiment Analyst).",
    )
    description: str = Field(
        description="Description of the analyst's focus, expertise, and analytical approach.",
    )
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

class AnalystTeam(BaseModel):
    analysts: List[FinancialAnalyst] = Field(
        description="Team of financial analysts with diverse specializations.",
    )

class GenerateAnalystsState(TypedDict):
    stock_symbol: str # Stock symbol to analyze
    max_analysts: int # Number of analysts
    human_analyst_feedback: str # Human feedback
    analysts: List[FinancialAnalyst] # Generated analysts